# IPython interface for running GPU DSM + fitting on cluster

In [1]:
from dsm_tools import ssh_init, job_start, job_done, job_transfer_clean, uuid, os, server, username, password, Calculation, CalculationStar, CalculationLinear
from fdt_fit import fdt_fit, fdt, fdtvec
import numpy as np
import matplotlib.pyplot as plt

In [0]:
nChains = 5000
CDtoggle = 0
simTime = 100000
num_gpu = 2

In [2]:
x = CalculationLinear(30, 0, 5000, 2, 100000)

In [3]:
x.token

In [4]:
x.calc()

In [6]:
x.check_self()

In [7]:
x.plot_fit_results()

In [8]:
fig = plt.figure(figsize=(24, 6))

ax1 = fig.add_subplot(131)

ax1.set_title("Entanglement lifetime distribution")
ax1.set_xlabel(r'$t/\tau_c$')
ax1.set_ylabel(r'$f_d(t)$')

ax1.scatter(x.fdt_x, x.fdt_y, c='r', label=r'$f_d(t)$')
ax1.plot(x.fdt_x, fdtvec(time=x.fdt_x, params=np.append(x.lambdaArr, x.gArr)), c='b', label=r'MMM fit')
leg = ax1.legend()
ax1.set_xscale('log')
ax1.set_yscale('log')

ax2 = fig.add_subplot(132)

ax2.set_title(r'$p^{cr}\left(\tau\right)$')
ax2.set_xlabel(r'$\lambda$')
ax2.set_ylabel(r'$g$')

ax2.scatter(x.lambdaArr, x.gArr, c='b')

leg = ax2.legend()
ax2.set_xscale('log')
ax2.set_yscale('log')
plt.xlim(xmin=0.5*min(x.lambdaArr), xmax=2*max(x.lambdaArr))
plt.ylim(ymin=0.5*min(x.gArr), ymax=2*max(x.gArr))

ax3 = fig.add_subplot(133)

ax3.set_title(r'$p^{eq}\left(\tau\right)$')
ax3.set_xlabel(r'$\lambda$')
ax3.set_ylabel(r'$g$')

ax3.scatter(x.lambdaArr, np.multiply(x.lambdaArr, x.gArr)/np.dot(x.lambdaArr, x.gArr), c='b')

leg = ax3.legend()
ax3.set_xscale('log')
ax3.set_yscale('log')
plt.xlim(xmin=0.5*min(x.lambdaArr), xmax=2*max(x.lambdaArr))
plt.ylim(ymin=0.5*min(np.multiply(x.lambdaArr, x.gArr)/np.dot(x.lambdaArr, x.gArr)), ymax=2*max(np.multiply(x.lambdaArr, x.gArr)/np.dot(x.lambdaArr, x.gArr)))

plt.show()

In [281]:
y = CalculationStar(3, [6, 6, 6], 0, 5000, 2, 100000)

In [282]:
y.token

In [208]:
y.calc()

In [213]:
y.check_self()

### Mix star and linear relaxation spectra

In [252]:
xWeight = 0.2
yWeight = 0.8

In [253]:
fig = plt.figure(figsize=(8, 6))

ax3 = fig.add_subplot(111)

ax3.set_title(r'$p^{eq}\left(\tau\right)$')
ax3.set_xlabel(r'$\lambda$')
ax3.set_ylabel(r'$g$')

ax3.scatter(np.append(x.lambdaArr, y.lambdaArr), np.append(np.multiply(x.gArr, x.lambdaArr)/np.dot(x.gArr, x.lambdaArr) * xWeight, np.multiply(y.gArr, y.lambdaArr)/np.dot(y.gArr, y.lambdaArr) * yWeight), c='b')

leg = ax3.legend()
ax3.set_xscale('log')
ax3.set_yscale('log')
plt.xlim(xmin=0.5*min(np.append(x.lambdaArr, y.lambdaArr)), xmax=2*max(np.append(x.lambdaArr, y.lambdaArr)))
plt.ylim(ymin=0.5*min(np.append(np.multiply(x.gArr, x.lambdaArr)/np.dot(x.gArr, x.lambdaArr) * xWeight, np.multiply(y.gArr, y.lambdaArr)/np.dot(y.gArr, y.lambdaArr) * yWeight)), ymax=2*max(np.append(np.multiply(x.gArr, x.lambdaArr)/np.dot(x.gArr, x.lambdaArr) * xWeight, np.multiply(y.gArr, y.lambdaArr)/np.dot(y.gArr, y.lambdaArr) * yWeight)))

plt.show()

In [90]:
x2=x
x2.CDtoggle = 1

In [91]:
x2.CDtoggle